In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphEmbedderObjective import SubgraphEmbeddingObjective, \
    SubgraphMatchingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist
from SourceCodeTools.models.graph.train.objectives.SCAAClassifierObjetive import SCAAClassifierObjective


import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "./examples/new_scaa/with_ast"
subgraph_partition = join(data_path, "subgraph_partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

In [4]:
unpersist(subgraph_partition)

,id,train_mask,val_mask,test_mask
0,4062,True,False,False
1,2469,True,False,False
2,976,True,False,False
3,2856,True,False,False
4,5022,True,False,False
...,...,...,...,...
5533,3164,True,False,False
5534,1848,True,False,False
5535,1858,True,False,False
5536,1987,True,False,False


In [5]:
unpersist(join(data_path, "common_edges.json.bz2"), nrows=10)

,id,type,source_node_id,target_node_id,file_id,mentioned_in
0,0,subword,438413,103096,4062,NaN
1,1,name,103096,339947,4062,118342.0
2,2,name_rev,339947,103096,4062,118342.0
3,3,names,339947,552237,4062,118342.0
4,4,names_rev,552237,339947,4062,118342.0
...,...,...,...,...,...,...
285,2960995,prev,153216,177156,4107,343723.0
286,2960996,defined_in_module,153216,343723,4107,343723.0
287,2960997,defined_in_module_rev,343723,153216,4107,343723.0
288,2960998,subword,438438,70188,4107,NaN


In [6]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    measure_scores=True,
    dilate_scores=1,
    epochs=100,
)

In [7]:
save_config(config, "var_misuse_tiny.yaml")

In [8]:
config

{'DATASET': {'data_path': './examples/new_scaa/with_ast',
  'train_frac': 0.9,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': None,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': './examples/new_scaa/with_ast/subgraph_partition.json.bz2'},
 'TRAINING': {'model_output_dir': './examples/new_scaa/with_ast',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 100,
  'embedding_table_size': 200000,
  'epochs': 100,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgraph_clf',
  'save_each_epoch': False,
  'save_checkpoints': True,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_ns': F

# Create Dataset

In [9]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

/home/alina/workspace/scaa/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.nodes = self.nodes.append(new_nodes, ignore_index=True)
/home/alina/workspace/scaa/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.edges = self.edges.append(new_edges, ignore_index=True)


# Declare target loading function (labels)

In [10]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "user"]].rename({"id": "src", "user": "dst"}, axis=1)

In [11]:
load_labels()

,src,dst
0,4062,milan_tom
1,2469,ajunwalker1
2,976,Jwang
3,2856,carlossless
4,5022,trojan6120
...,...,...
5533,3164,eth42
5534,1848,Sestren
5535,1858,Shanshan33
5536,1987,StatujaLeha


One or several objectives could be used

In [12]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="SCAAMatching",
                objective_class=SCAAClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

In [13]:
# %tensorboard --logdir data
from tensorboard import program

tracking_address = './examples/new_scaa/' # the path of your log file.

tb = program.TensorBoard()
tb.configure(argv=[None, '--logdir', tracking_address])
url = tb.launch()
print(f"Tensorflow listening on {url}")


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784



Tensorflow listening on http://localhost:6006/


In [14]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Precompute Target Embeddings:   0%|                                                                                     | 0/44 [00:00<?, ?it/s]

Number of nodes 758247


Epoch 0:   0%|                                                                                                          | 0/42 [00:04<?, ?it/s]


RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor